# INTELIGENCIA ARTIFICIAL: EB Ejercicios Tema 7: SISTEMAS DE RECOMENDACIÓN 
## EJERCICIO: Normalización de las valoraciones
**Implementar** un sistema de recomendación de filtrado colaborativo como el visto en la EPD6. Después, **implementar** el algoritmo de normalización de las valoraciones ($mean$ $normalization$). 




In [1]:
import numpy as np
import pandas as pd
import scipy.optimize as opt
import scipy.io as sio

# A) SISTEMA DE RECOMENDACIÓN BASADO EN FILTRADO COLABORATIVO

## 1) Cargar los datos de entrada
Serán los mismos que los usados en la EPD6: 
*   "ex8_movies.mat": Valoraciones de las 1682 películas por los 943 usuarios.
*   "ex8_movieParams.mat": Parámetros de un modelo pre-entrenado.

In [2]:
movies = sio.loadmat("ex8_movies.mat")
Y = movies['Y'] 
R = movies['R']
n_items = Y.shape[0]
n_users  = Y.shape[1]
print("Número de películas: ", Y.shape[0], " número de usuarios: ", Y.shape[1])
#print("Número de películas: ", R.shape[0], " número de usuarios: ", R.shape[1]) # Sería lo mismo que la sentencia anterior

print("\nY contiene las puntuaciones/valoraciones de 1-5 de las n_i películas y los n_u usuarios.")
print("\t Y es un ", type(Y), " con dimensiones: ", Y.shape)
print("\nR indica si existe o no valoración de un usuario para una película.")
print("\t R es un ", type(R), " con dimensiones: ", R.shape)

print('\nMedia de las valoraciones de la primera película(Toy Story): ', Y[0, np.where(R[0, :] == 1)[0]].mean(), "/5\n")

###

params_data = sio.loadmat("ex8_movieParams.mat")
X = params_data['X']
Theta = params_data['Theta']
Theta = Theta.T
features = X.shape[1] # Sería lo mismo que Theta.shape[0]
print("****    *****\n\nEl número de características de los ítems (películas) es : ", features)

print("\nX contiene las características preentrenadas basadas en el contenido de las películas.")
print("\t X es un ", type(X), " con dimensiones: ", X.shape)
print("\nTheta contiene los parámetros preentrenados de preferencia de nuestros usuarios.")
print("\t Theta es un ", type(Theta), " con dimensiones: ", Theta.shape)

Número de películas:  1682  número de usuarios:  943

Y contiene las puntuaciones/valoraciones de 1-5 de las n_i películas y los n_u usuarios.
	 Y es un  <class 'numpy.ndarray'>  con dimensiones:  (1682, 943)

R indica si existe o no valoración de un usuario para una película.
	 R es un  <class 'numpy.ndarray'>  con dimensiones:  (1682, 943)

Media de las valoraciones de la primera película(Toy Story):  3.8783185840707963 /5

****    *****

El número de características de los ítems (películas) es :  10

X contiene las características preentrenadas basadas en el contenido de las películas.
	 X es un  <class 'numpy.ndarray'>  con dimensiones:  (1682, 10)

Theta contiene los parámetros preentrenados de preferencia de nuestros usuarios.
	 Theta es un  <class 'numpy.ndarray'>  con dimensiones:  (10, 943)


## 2) Función coste del filtrado colaborativo con regularización
Usar el subconjunto de datos formado por los 4 primeros usuarios, 5 primeras películas y 3 primeros atributos y también para todos los datos del fichero "ex8_movies.mat".

In [3]:
def cofiCostFuncReg(params, Y, R, num_features, lambda_param):
    num_movies, num_users = Y.shape
    
    # Desenrollar parámetros
    X = np.reshape(params[:num_movies*num_features], (num_movies, num_features), 'F')
    Theta = np.reshape(params[num_movies*num_features:], (num_features, num_users), 'F')
    J = 0
    
    error = np.multiply(np.dot(X, Theta) - Y, R)
    squared_error = np.power(error, 2)
    J = 1/2 * np.sum(squared_error)
    
    # Regularización
    J += lambda_param/2 * (np.sum(np.power(Theta, 2)) + (lambda_param/2)* np.sum(np.power(X, 2)))
    
    return J

In [4]:
lambda_param = 1.5

# Subconjunto de datos
sub_users = 4
sub_movies = 5
sub_features = 3

X_sub = X[:sub_movies, :sub_features]
Theta_sub = Theta[:sub_features, :sub_users]
Y_sub = Y[:sub_movies, :sub_users]
R_sub = R[:sub_movies, :sub_users]

params_sub = np.hstack((np.ravel(X_sub, order='F'), np.ravel(Theta_sub, order='F')))

J_reg_sub = cofiCostFuncReg(params_sub, Y_sub, R_sub, sub_features, lambda_param)
print("\nPara el subconjunto seleccionado J_reg debe ser cercano a 31.34: ",J_reg_sub)

# Todos los datos
params = np.hstack((np.ravel(X, order='F'), np.ravel(Theta, order='F')))
J_reg = cofiCostFuncReg(params, Y, R, features, lambda_param)
print("\nPara todos los datos J_reg debe ser cercano a 34821.70: ",J_reg)


Para el subconjunto seleccionado J_reg debe ser cercano a 31.34:  30.0756100583271

Para todos los datos J_reg debe ser cercano a 34821.70:  33958.21124286916


## 3) Gradiente del filtrado colaborativo con regularización
Usar el subconjunto de datos formado por los 4 primeros usuarios, 5 primeras películas y 3 primeros atributos y también para todos los datos del fichero "ex8_movies.mat".

In [5]:
def cofiGradientFuncReg(params, Y, R, num_features, lambda_param):
    num_movies, num_users = Y.shape
    
    # Desenrollar parámetros
    X = np.reshape(params[:num_movies*num_features], (num_movies, num_features), 'F')
    Theta = np.reshape(params[num_movies*num_features:], (num_features, num_users), 'F')
    
    X_grad = np.zeros(X.shape)
    Theta_grad = np.zeros(Theta.shape)
    
    # Coste
    error = np.multiply(np.dot(X, Theta) - Y, R)
    X_grad = np.dot(error, Theta.T)
    Theta_grad = np.dot(X.T, error)
    X_grad += lambda_param * X
    Theta_grad += lambda_param * Theta
    
    # Desenrollar gradientes
    grad = np.hstack((np.ravel(X_grad, order="F"), np.ravel(Theta_grad,order="F")))

    return grad

In [6]:
lambda_param = 1.5

# Subconjunto de datos
sub_users = 4
sub_movies = 5
sub_features = 3

X_sub = X[:sub_movies, :sub_features]
Theta_sub = Theta[:sub_features, :sub_users]
Y_sub = Y[:sub_movies, :sub_users]
R_sub = R[:sub_movies, :sub_users]

params_sub = np.hstack((np.ravel(X_sub, order='F'), np.ravel(Theta_sub, order='F')))

grad_reg_sub = cofiGradientFuncReg(params_sub, Y_sub, R_sub, sub_features, lambda_param)
print("\nGradiente para el subconjunto seleccionado: ",grad_reg_sub)

# Todos los datos
params = np.hstack((np.ravel(X, order='F'), np.ravel(Theta, order='F')))
grad_reg = cofiGradientFuncReg(params, Y, R, features, lambda_param)
print("\nGradiente para todos los datos: ",grad_reg)


Gradiente para el subconjunto seleccionado:  [ -0.95596339   0.60308088   0.12985616   0.29684395   0.60252677
   6.97535514   2.77421145   4.0898522    1.06300933   4.90185327
  -0.10861109   0.25839822  -0.89247334   0.66738144  -0.19747928
 -10.13985478   2.10136256  -6.76563628  -2.29347024   0.48244098
  -2.99791422  -0.64787484  -0.71820673   1.27006666   1.09289758
  -0.40784086   0.49026541]

Gradiente para todos los datos:  [-4.68881319 -2.63803761 -2.16863787 ... -4.63438191  3.72934198
  1.85226694]


## 4) Añadir nuevo usuario sin valoración
Añadir un nuevo usuario sin valoración consiste en:
*   Añadir al array $Y$ un nuevo array vacío con 1 columna y tantas filas como ítems tengamos. Usar $np.empty([n_{filas}, n_{columnas}])$.
*   Añadir al array $R$ un nuevo array de ceros con 1 columna y tantas filas como ítems tengamos. Usar $np.zeros((n_{filas}, n_{columnas}))$.
*   Aumentar el número de usuarios en 1.


In [7]:
# Añadir nuevo usuario del que no conocemos ninguna valoración
null_array = np.zeros((n_items, 1))
Y = np.append(Y, null_array, axis=1) # Append siempre añade al final del array. Axis=1 para añadir los valores como columna

zeros_rating = np.zeros((n_items, 1))
R = np.append(R, zeros_rating, axis=1) # Append siempre añade al final del array. Axis=1 para añadir los valores como columna

n_users = n_users+1 # IMPORTANTE!

## 5) Inicializar X y $\Theta$ de forma random
Inicializar de forma random con valores pequeños tanto la matriz X como la matriz Theta para todo el conjunto de datos. Usar la función $np.random.rand()$ indicando las dimensiones en los parámetros de entrada.


In [8]:
# Inicializar Theta y X con valores random pequeños
X = np.random.rand(n_items, features)
Theta = np.random.rand(n_users, features)

## 6) Normalización de las valoraciones

### **Normalización de las valoraciones en sistemas de recomendación:**###

Cuando tengamos un nuevo usuario sin valoraciones (r(i,j)=0 para todos los ítems) ni preferencias, solo podremos minimizar en la función coste del filtrado colaborativo la parte correspondiente a la regularización de $\Theta$. Por lo que los parámetros $\Theta$ para el usuario $j$ acabarán siendo muy cercanos a 0. Y por tanto, todas las predicciones de valoración de los ítems del nuevo usuario serán cercanas a 0. 

Una solución a este problema es normalizar las valoraciones por película que tengamos. Pasos:
1.   Crear array $Y_{mean}$ con dimensiones: [n$_i$, 1] donde para cada ítem (fila) tendremos la media de las valoraciones.
2.   Crear array $Y_{norm}$ con dimensiones: [n$_i$, n$_u$] donde a la valoración de cada película (en $Y$) le restaremos su media correspondiente (en $Y_{mean}$). Si desconocemos la valoración, la dejaremos sin conocer.
\begin{equation}
Y_{norm} = Y - Y_{mean}
\end{equation}
3.   Haremos las predicciones de las valoraciones por ítem (usando como siempre la hipótesis) y le sumaremos la media correspondiente ($Y_{mean}$). Es decir, la predicción de la valoración del usuario $j$ al ítem $i$ sería:
\begin{equation}
(\Theta^{(j)})^{T} x^{i} + Y_{mean}^{i}
\end{equation}
De forma vectorizada, las predicciones se pueden representar como:
\begin{equation}
 X · \Theta + Y_{mean}
 \end{equation}
4.   Por tanto, si tenemos un usuario sin ninguna valoración, las predicciones de sus valoraciones serán muy cercanas a las medias.

La normalización de las valoraciones puede usarse también como parte del preprocesado de los datos.

In [9]:
def normalizacion (n_items, n_users, R, Y):
  # Inicialización con ceros de Ymean y Ynorm con dimensiones adecuadas
  Ymean = np.zeros((n_items, 1))
  Ynorm = np.zeros((n_items, n_users))
  # Para cada ítem
  for i in range(n_items):
      idx = np.where(R[i, :] == 1)[0]
      Ymean[i] = np.mean(Y[i, idx])
      Ynorm[i, idx] = Y[i, idx] - Ymean[i]
  print("Mean Y matrix normalized: ", Ynorm.mean())
  return Ymean, Ynorm

In [10]:
# Normalización con usuario nuevo 
Ymean, Ynorm = normalizacion(n_items, n_users, R, Y)

Mean Y matrix normalized:  6.178285186614429e-19


## 7) Función optimizadora y predicción
Usar la función optimizadora fmin_cg de la librería scipy.optimize. Usar las versiones regularizadas de la función coste y la función que calcula el gradiente. Utilizar el vector $Y$ normalizado y los parámetros $X$ y $\Theta$ inicializados de forma random.  

Una vez obtenidos los parámetros $X$ y $\Theta$ que optimizan la función coste, calcular las predicciones considerando que hemos normalizado. Almacenar en la variable $my_{preds}$ las predicciones correspondientes al nuevo usuario que hemos añadido (como es el último del array, podemos acceder a él seleccionando la columna "-1").

In [11]:
# Desenrollar
lambda_param = 0.1
g_tol = 0.001
params_rnd = np.hstack((np.ravel(X, order='F'), np.ravel(Theta, order='F')))

# Función optimizadora
fmin = opt.fmin_cg(gtol=g_tol, maxiter=300,f=cofiCostFuncReg, x0=params_rnd, fprime=cofiGradientFuncReg, args=(Ynorm, R, features, lambda_param))

# Enrollar los parámetros optimizados
X = np.reshape(fmin[:n_items*features], (n_items, features), order='F')
Theta = np.reshape(fmin[n_items*features:], (n_users, features), order='F')

         Current function value: 23332.785419
         Iterations: 300
         Function evaluations: 441
         Gradient evaluations: 441


c:\Users\danit\anaconda3\envs\entornoIA2425\lib\site-packages\scipy\optimize\_optimize.py:1659: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_cg(f, x0, args, fprime, callback=callback, c1=c1, c2=c2,


In [12]:
# Predicciones con normalización
predictions = np.dot(X, Theta.T) + Ymean
display(predictions)

# Me quedo solo con el último usuario: el que he añadido nuevo
my_preds = predictions[:, -1]
display(my_preds)

array([[4.33434783, 3.92830918, 4.6608644 , ..., 3.99453985, 4.13754691,
        3.90209303],
       [3.79725389, 3.72180555, 4.71795211, ..., 3.60002316, 3.15648666,
        2.22421654],
       [2.96150795, 3.74753444, 3.94485256, ..., 4.22206009, 1.34541687,
        2.726379  ],
       ...,
       [2.04861347, 2.04359759, 2.02963014, ..., 1.98381064, 2.03067833,
        2.18050461],
       [3.01440905, 3.06069226, 3.04645311, ..., 3.01925073, 3.01579319,
        2.98960574],
       [2.97372566, 3.06149023, 3.00177347, ..., 3.07479575, 3.02021226,
        3.18317384]])

array([3.90209303, 2.22421654, 2.726379  , ..., 2.18050461, 2.98960574,
       3.18317384])

## 8) Recomendamos las 10 películas con valoración predicha mayor para el nuevo usuario
Leer los datos correspondientes a las películas desde "movie_ids.txt".
Para recomendar las 10 películas con predicción de valoración más alta, usar la sentencia $np.argsort()$ que ordena de menor a mayor. Después para conseguir que ordene de mayor a menor usar $[::-1]$.

In [13]:
# Leemos el fichero con los ids y nombres de las películas
movie_idx = {}
f = open('movie_ids.txt',encoding = 'ISO-8859-1')
for line in f:
    tokens = line.split(' ')
    tokens[-1] = tokens[-1][:-1]
    movie_idx[int(tokens[0]) - 1] = ' '.join(tokens[1:])

In [14]:
idx = np.argsort(my_preds, axis=0)[::-1] # Ordenar por las predicciones de menor a mayor y coger sus índice. [::-1] significa que le damos la vuelta a la salida: de mayor a menor

print("Top 10 movie predictions:")
# Imprimir las 10 películas con predicción de valoración del nuevo usuario más altas
for i in range(10):
      j = int(idx[i])
      print('Predicted rating of {0} for movie {1}.'.format(str(float(my_preds[j])), movie_idx[j]))

Top 10 movie predictions:
Predicted rating of 11.868202954983179 for movie Freeway (1996).
Predicted rating of 11.69211802941491 for movie Burnt By the Sun (1994).
Predicted rating of 11.690230383701408 for movie Bread and Chocolate (Pane e cioccolata) (1973).
Predicted rating of 10.453704600621816 for movie Ponette (1996).
Predicted rating of 9.925086921832772 for movie Primary Colors (1998).
Predicted rating of 9.863801470114133 for movie Grace of My Heart (1996).
Predicted rating of 9.852603778356864 for movie Big Blue, The (Grand bleu, Le) (1988).
Predicted rating of 9.829490009347312 for movie Love Affair (1994).
Predicted rating of 9.758863913035487 for movie Live Nude Girls (1995).
Predicted rating of 9.597860125915398 for movie Strawberry and Chocolate (Fresa y chocolate) (1993).
